In [1]:
import pandas as pd
import numpy as np

In [2]:
# Utility methods
def most_frequent(list_of_strings):
    # Create an empty dictionary
    word_count = {}
    # Loop through the list of strings
    for string in list_of_strings:
        # Split the string into words
        words = string.split()
        # Loop through the words and update the count in the dictionary
        for word in words:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

    # Find the word with the highest count
    most_frequent_word = max(word_count, key=word_count.get)
    return most_frequent_word

def remove_whitespace(input_string):
    return "".join(input_string.split())

def has_substring(input_string, substring):
    return substring in input_string
    
def parse_amount(amount_string):
    return float(amount_string.replace(",", ""))

In [7]:
def feats_from_credit_report_json (jasonfile):
    #expects a json file 
    df_data = pd.read_json("credit_report_sample.json")
    tmp = df_data['data']
    data_col = list(tmp)
    # use the data col onwards for feat extraction
    good_acc_list = []
    bad_acc_list = []
    occ_list = []
    cred_list = []
    outstan_debt_list = []
    gender_list = []
    nationality_list = []
    amt_overdue_list = []

    for app in data_col:
        good = 0
        bad = 0
        for item in app['consumerfullcredit']['accountrating']:
        #print(items['consumerfullcredit']['accountrating'])
        #print("*"*25)
            if ('good' in item):
                good += int(app['consumerfullcredit']['accountrating'][item])
            if ('bad' in item):
                bad += int(app['consumerfullcredit']['accountrating'][item])
        good_acc_list.append(good)
        bad_acc_list.append(bad)

        tmp_occ_list = []
        for item in app['consumerfullcredit']["employmenthistory"]:
            tmp_occ_list.append(remove_whitespace(item['occupation']))
        occ_list.append(most_frequent(tmp_occ_list))

        rating =0
        outstand_debt = 0
        dishonor_amt = 0
        rating = int(app['consumerfullcredit']["creditaccountsummary"]['rating'])
        outstan_debt = parse_amount(app['consumerfullcredit']["creditaccountsummary"]['totaloutstandingdebt'])
        cred_list.append(rating)
        outstan_debt_list.append(outstan_debt)

    
        gender = app['consumerfullcredit']['personaldetailssummary']['gender']
        gender_list.append(gender)
        nationality = app['consumerfullcredit']['personaldetailssummary']['nationality']
        nationality_list.append(nationality)
        amt_overdue = 0
        for item in app['consumerfullcredit']['accountmonthlypaymenthistory']:
            if (item.get('amountoverdue',None)):
                amt_overdue += parse_amount(item['amountoverdue'])
        amt_overdue_list.append(amt_overdue)        

    df_data['good_accounts'] = good_acc_list
    df_data['bad_accounts'] = bad_acc_list
    df_data['occupation'] = occ_list
    df_data['credit_rating'] = cred_list
    df_data['outstanding_debt_amount'] = outstan_debt_list
    df_data['gender'] = gender_list
    df_data['nationality'] = nationality_list
    df_data['amount_overdue_total'] = amt_overdue_list
    df_data = df_data.drop('data', axis=1)

    return df_data


In [8]:
jsonfile = "credit_report_sample.json"
credit_df = feats_from_credit_report_json (jsonfile)
credit_df.head()

,application_id,good_accounts,bad_accounts,occupation,credit_rating,outstanding_debt_amount,gender,nationality,amount_overdue_total
0,9711360,7,0,PUBLICSERVANTS,13,105435.0,Male,Nigeria,24042.07
1,9714953,17,0,DOCTOR,2,294770.0,Female,Nigeria,0.00
2,9714978,2,1,STUDENT,109,110919.0,Female,Nigeria,12000.00
